In [1]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
#Initializing mediapipe pose estimation
mp_pose = mp.solutions.pose

pose_img = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [3]:
#Drawing pose landmarks on image
def detectPose(img, draw=True):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose_img.process(imgRGB)
    if results.pose_landmarks:
        if draw:
            mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    return img

In [6]:
#Video Inference
cap = cv2.VideoCapture('market.mp4')
if cap.isOpened() == False:
    print('Error')
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
video_codec = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', video_codec, 20.0, (frame_width, frame_height))
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame = detectPose(frame)
        total_frames +=1
        fps_end_time = datetime.datetime.now()
        time_diff = fps_end_time - fps_start_time
        if time_diff.seconds == 0:
            fps = 0
        else:
            fps = (total_frames/time_diff.seconds)
        
        #Saving the video
        out.write(frame)
        fps_text = "FPS: {}".format(fps)
        frame = cv2.putText(frame, fps_text, (5,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (125, 0, 225), 1)
        cv2.imshow('Video Inference', frame)
        
    else:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()